# 2. Full Bangladesh dataset preprocessing

 Acceptance criteria for Bangladesh dataset
 
- if there is a continuous gap of 3 years -> split

- any sudden shifts indicating changes to the well location and depth -> split


_(inludes manual preprocessing steps supported by an expert examination of time-series plots)_

In [1]:
import geopandas as gpd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import rioxarray as rxr
import geopandas as gpd

import utils.outliers_helpers as ohp
import utils.changepoint_helpers as chp
from matplotlib.backends.backend_pdf import PdfPages

In [2]:
MIN_VALID_POINTS_PER_SEGMENT = 60 # Minimum non-NaN points for a segment to be processed
GAP_THRESHOLD_MONTHS = 36
root_path = './data/Bangladesh/' 

#### 1. Read files 

In [3]:
info_file = pd.read_csv(f'{root_path}/target/well_info.csv')
info_file = gpd.GeoDataFrame(
    info_file, geometry=gpd.points_from_xy(info_file.Longitude, info_file.Latitude), crs="EPSG:4326"
)

ts_file = pd.read_csv(f'{root_path}/target/raw_data.csv')

storage_co_da = rxr.open_rasterio(f"{root_path}/BGD_storage_coefficient.tif")

In [4]:
# filter the relevant dates
ts_file['Date'] = pd.to_datetime(ts_file['Date'], format='%d/%m/%Y')
# Correct years that are mistakenly in the 2000s
ts_file['Date'] = ts_file['Date'].apply(lambda x: x.replace(year=x.year - 100) if x.year > 2025 else x)

ts_file = ts_file.set_index('Date')

In [5]:
ts_file

,SY101,FA062,RA-63,DI077,BO033,PT005,DI058,RAJ-A-4,FA012,CT017,...,DI068S,MY041,KTS009,PAS02,RJS024,RJS026,SY096,PAS01,RJ083,RJ119
Date,,,,,,,,,,,,,,,,,,,,,
1961-01-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1961-02-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1961-03-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1961-04-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1961-05-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-08-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-10-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### 2. Preprocess
outlier detection: remove outliers

large gap detection: split ts

change detection: split ts

In [6]:
segment_means = {}
outlier_detected = {}
well_changed = {}

ts_file_gws = {}
ts_file_gwl = {}  
ts_file_bgl = {}  

for well in ts_file.columns:
    if well == 'Date': continue

    ts_info = info_file.loc[info_file['OldID'] == well].head(1)

    # skip wells that have no corresponding information
    if ts_info.empty or np.isnan(ts_info.Longitude).item():
        continue

    # skip wells that have extremely unreliable/messy time series (after visual inspection)
    # some of them could have been impacted by flooding that damaged the measuring equipment 
    if well in ['CT050', 'BA026', 'MY081']:
        continue

    # --- Gap Detection and Splitting ---
    well_series_raw = ts_file[well].copy()
    well_series_raw = well_series_raw - ts_info.ParaHeight.item() + ts_info.RefLevel.item()
    valid_points = well_series_raw.dropna()

    if valid_points.empty:
        print(f"Well {well} has no valid data points. Skipping.")
        continue

    valid_indices = valid_points.index
    segments_to_process = []

    if len(valid_indices) > 1:
        # Calculate month differences between consecutive valid points
        month_number = (valid_indices.year * 12 + valid_indices.month).to_series(index=valid_indices)
        month_diff = month_number.diff()

        # Find the start dates of segments following a large gap
        split_points_start_dates = valid_indices[month_diff > GAP_THRESHOLD_MONTHS]

        # Define segment boundaries
        current_start_date = valid_indices.min() # Start from the first valid point
        for split_date in split_points_start_dates:
            # Define the end date for the current segment (inclusive)
            # Find the last valid index strictly before the split_date
            segment_end_date = valid_indices[valid_indices < split_date].max() + pd.DateOffset(months=1)
            segment = well_series_raw.loc[current_start_date:segment_end_date]
            if not segment.dropna().empty: # Ensure segment has data
                 segments_to_process.append(segment)
            current_start_date = split_date # Next segment starts at the split date

        # Add the final segment (from the last split point or the beginning to the end)
        last_segment = well_series_raw.loc[current_start_date:valid_indices.max()]
        if not last_segment.dropna().empty:
             segments_to_process.append(last_segment)

    elif len(valid_indices) == 1:
         segments_to_process.append(valid_points) # Only one point, treat as a segment
    else: # Should have been caught by valid_points.empty() check, but for safety
        continue

    # If no gaps were found, segments_to_process will contain the whole series (if valid)
    if not segments_to_process and not valid_points.empty:
         segments_to_process.append(well_series_raw)

    # --- Process Each Segment ---
    segment_count = len(segments_to_process)
    for i, segment in enumerate(segments_to_process):
        segment = segment.copy() # Work on a copy

        # Check for sufficient valid points in the segment
        if segment.dropna().shape[0] < MIN_VALID_POINTS_PER_SEGMENT:
            print(f"Well {well}, Segment {i+1}/{segment_count}: Too few valid points ({segment.dropna().shape[0]}). Skipping segment.")
            if well not in ['RA-36', 'PA012', 'JA012']: # Don't discard data that should be stitched in postprocessing
                continue

        # Determine the column name for the processed segment
        segment_col_name = f"{well}_{i+1}" if segment_count > 1 else well

        # Find storage co for GWS conversion later
        try:
            st_co = storage_co_da.sel(y=ts_info['Latitude'].item(), x=ts_info['Longitude'].item(), method="nearest").item()
            if np.isnan(st_co) or st_co <= 0: # Handle invalid storage coefficient
                 print(f"Warning: Well {well}, Segment {i+1}: Invalid storage coefficient ({st_co}). Skipping segment.")
                 continue
        except Exception as e:
            print(f"Warning: Well {well}, Segment {i+1}: Could not get storage coefficient at coords ({ts_info['Longitude'].item()}, {ts_info['Latitude'].item()}). Error: {e}. Skipping segment.")
            continue
            
        # --- Outlier Detection (on the segment GWL) ---
        segment_cleaned, outlier_found = ohp.remove_outliers(segment.copy())
        outlier_detected[segment_col_name] = outlier_found

        # --- Change Point Detection (on the cleaned segment GWL) ---
        well_changed[segment_col_name] = chp.detect_well_change(segment_cleaned.copy())
        # --- Add Time Series Segments defined by change points ---
        seg_ts, bps = well_changed[segment_col_name]
        if len(bps) > 0:
            start_idx = 0
            for i, bp in enumerate(bps):
                seg_col_name = f'{segment_col_name}_{i}'
                seg_cleaned, _ = ohp.remove_outliers(seg_ts[start_idx:bp].copy())
                if seg_cleaned.dropna().shape[0] < MIN_VALID_POINTS_PER_SEGMENT:
                    start_idx = bp
                    continue
                ts_file_gws[seg_col_name] = seg_cleaned * ( st_co / 100 )
                ts_file_gwl[seg_col_name] = seg_cleaned
                ts_file_bgl[seg_col_name] = ts_file_gwl[seg_col_name] + ts_info.ParaHeight.item() - ts_info.RefLevel.item()
                start_idx = bp
            final_seg_cleaned, _ = ohp.remove_outliers(seg_ts[start_idx:].copy())
            if final_seg_cleaned.dropna().shape[0] < MIN_VALID_POINTS_PER_SEGMENT:
                # I'm stitching these manually later
                if well not in ['SY074', 'MY084', 'BO038']:
                    continue
            ts_file_gws[f'{segment_col_name}_{i+1}'] = final_seg_cleaned * ( st_co / 100 )
            ts_file_gwl[f'{segment_col_name}_{i+1}'] = final_seg_cleaned
            ts_file_bgl[f'{segment_col_name}_{i+1}'] = ts_file_gwl[f'{segment_col_name}_{i+1}'] + ts_info.ParaHeight.item() - ts_info.RefLevel.item()
        else:        
            # --- Store Processed Segment ---
            # Add the processed segment to the results DataFrame, aligning index
            ts_file_gws[segment_col_name] = segment_cleaned * ( st_co / 100 )
            ts_file_gwl[segment_col_name] = segment_cleaned
            ts_file_bgl[segment_col_name] = ts_file_gwl[segment_col_name] + ts_info.ParaHeight.item() - ts_info.RefLevel.item()

ts_file_gws = pd.DataFrame(ts_file_gws, index=ts_file.index)
ts_file_gwl = pd.DataFrame(ts_file_gwl, index=ts_file.index)  
ts_file_bgl = pd.DataFrame(ts_file_bgl, index=ts_file.index)

print("Processing finished.")
print(f"Generated {ts_file_gws.shape[1]} processed time series segments.")

Well RA-36, Segment 2/2: Too few valid points (23). Skipping segment.
Well JE006, Segment 2/2: Too few valid points (41). Skipping segment.
Well DH111, Segment 1/2: Too few valid points (19). Skipping segment.
Well DI040, Segment 1/2: Too few valid points (44). Skipping segment.
Well PA012, Segment 2/2: Too few valid points (35). Skipping segment.
Well RJ052, Segment 1/2: Too few valid points (53). Skipping segment.
Well BA011, Segment 1/2: Too few valid points (40). Skipping segment.
Well DI046, Segment 1/2: Too few valid points (59). Skipping segment.
Well KT019, Segment 2/2: Too few valid points (28). Skipping segment.
Well CM033, Segment 2/2: Too few valid points (2). Skipping segment.
Well JE021, Segment 1/2: Too few valid points (14). Skipping segment.
Well TA037, Segment 2/2: Too few valid points (13). Skipping segment.
Well CT012, Segment 2/2: Too few valid points (1). Skipping segment.
Well RJ049, Segment 1/2: Too few valid points (41). Skipping segment.
Well DI068, Segment 1/

#### 3. Manual quality control
stitch, split, remove

In [7]:
### manual outlier removal
def remove_outliers_manually(mask, well):
    ts_file_bgl.loc[mask, well] = np.nan
    ts_file_gwl.loc[mask, well] = np.nan
    ts_file_gws.loc[mask, well] = np.nan

min_out = { 
    'RJ002_1': -14, 'RJ007': -12, 'SY057': -4, 'RJ061': -10, 'RJ080': -11, 'MY005': -5, 'MY020': -8, 'MY030': -12, 'CM028': -3.5, 'DH085': -6,
    'DH083': -6, 'RA-53': -6, 'KT039': -12, 'RA-50': -5, 'MY071': -6.5, 'KH016': -3.5, 'RJ127': -20, 'KH012': -3, 'BO022': -8.5, 'SY105': -6, 'MY013': -6,
    'JA015': -7, 'MY080_1': -8, 'PA061': -8, 'DH004': -12, 'DH091_1': -9, 'PT007': -1.4, 'RJ110': -9, 'RJ141_1': -10, 'NA024_2_0': -4.7,
    'SY025_1': -4.5, 'PA055_2': -12, 'RJ126_1': -3, 'BA030_1': -1, 'RJS23_1_0': -8
}

for well, threshold in min_out.items():
    mask = ts_file_bgl[ts_file_bgl[well] < threshold].index
    remove_outliers_manually(mask, well)

min_out_constr = {'RA-44': ['1990-01-01', -4], 'SY112_1': ['1990-01-01', -3], 'DH057_0': ['1990-01-01', -4],
                 'PT009_1': ['2008-01-01', -0.5]}

for well, (s_date, threshold) in min_out_constr.items():
    mask = ts_file_bgl[ts_file_bgl[well] < threshold][:s_date].index
    remove_outliers_manually(mask, well)

max_out_constr = {'DH015_1': ['2010-01-01', -63], 'CT008': ['2005-01-01', 1], 'KH020': ['1980-01-01', -0.8], 'KH021': ['1980-01-01', -0.8],
                  'DH002_0': ['2010-01-01', -18], 'SY072_1_0': ['1990-01-01', -1], 'SY040_1': ['2015-01-01', -10], 'CT013_2_1': ['1992-01-01', -2], 
                  'DI030': ['1993-01-01', -0.2], 'SY068_2_1': ['1988-01-01', -0.5], 'SY044_0': ['1987-01-01',-2], 'SY035_1': ['1985-01-01', -0.5], 
                  'RJ105_1_0': ['1992-01-01', -9], 'RA-30_3': ['2010-01-01', -2], 'SY031_1_1': ['1990-01-01', 0], 'RJS29_1_2': ['1994-01-01', -10]
                 }

for well, (s_date, threshold) in max_out_constr.items():
    mask = ts_file_bgl[ts_file_bgl[well] > threshold][s_date:].index
    remove_outliers_manually(mask, well)

remove_range = {'FA058_2': ['1998-01-01', '2002-01-01'],
               'CT004_1': ['1982-06-01', '1985-01-01'] }

for well, (s_date, e_date) in remove_range.items():
    mask = ts_file_bgl[well][s_date:e_date].index
    remove_outliers_manually(mask, well)

In [8]:
# # stitching
stitch_ids = ['SY075', 'RJ013', 'BO029', 'RJ092', 'MY073', 'KT047', 'CT011', 'RJ106', 
              'MY030', 'RJ142', 'SY056', 'BO005', 'RJ002', 'RJ005', 'KT010', 'CT034', 
              'JA024', 'RJ015', 'JE036', 'RJ006', 'JE020', 'JE046', 'RA-16', 'BO009', 
              'CM006', 'KH009', 'JES08', 'JES11', 'RA-01', 'RJ008', 'JE013', 'BO010',
              'BO007', 'MY012', 'MY024', 'JE029', 'KT037', 'KT008', 'PA016', 'TA026',
              'RJ104', 'RJ089', 'BA009', 'DI057', 'PA009', 'RJ045', 'RJ014', 'CM010',
              'FA035', 'KH006', 'DI041', 'JE045', 'CT014', 'RA-49', 'MY084', 'DI091',
              'KT015', 'JA002', 'RJ141', 'RJ125', 'DH096', 'RJ133', 'KT046', 'FA033',
              'FA064', 'DH127', 'SY074', 'PA012', 'RA-36', 'DI019', 'DH110', 'SY093',
              'CT029', 'CT037', 'BO038', 'DI058', 'RA-66', 'RJ058', 'KT027', 'KT048',
              'RJ009', 'SY011', 'RJ081', 'RJ043', 'RJ122', 'RJ075', 'PA004', 'SY078',
              'RJ087', 'RJ032', 'DI048', 'DH015', 'KT004', 'JE060', 'RJ131', 'DH087',
              'RJ116', 'SY076', 'RJ139', 'BA029', 'NA018', 'DH091', 'TA003', 'DI055',
              'RJ012', 'JE050', 'JA009', 'SY018', 'KT028', 'CM059', 'MY043', 'SY045',
              'DH065', 'CM063', 'FA033', 'RJ027', 'SY099', 'RA-75', 'BA016', 'SY094',
              'CT027', 'TA031', 'KH011', 'SY097', 'PAS03', 'DI030']

for stitch_id in stitch_ids:
    parts_bgl = ts_file_bgl.filter(like=stitch_id, axis=1)
    parts_gwl = ts_file_gwl.filter(like=stitch_id, axis=1)
    parts_gws = ts_file_gws.filter(like=stitch_id, axis=1)

    ts_file_bgl = ts_file_bgl.drop(parts_bgl.columns, axis=1)
    ts_file_gwl = ts_file_gwl.drop(parts_gwl.columns, axis=1)
    ts_file_gws = ts_file_gws.drop(parts_gws.columns, axis=1)

    ts_file_bgl[stitch_id] = parts_bgl.bfill(axis=1).iloc[:, 0]
    ts_file_gwl[stitch_id] = parts_gwl.bfill(axis=1).iloc[:, 0]
    ts_file_gws[stitch_id] = parts_gws.bfill(axis=1).iloc[:, 0]

# stitching 2 first parts
stitch2_ids = ['PA020', 'DI051', 'SY055', 'SY048', 'CT022', 'NA006', 'RJ076']
for stitch2_id in stitch2_ids:
    parts_bgl = ts_file_bgl.filter(like=stitch2_id, axis=1).iloc[:,:2]
    parts_gwl = ts_file_gwl.filter(like=stitch2_id, axis=1).iloc[:,:2]
    parts_gws = ts_file_gws.filter(like=stitch2_id, axis=1).iloc[:,:2]
    
    ts_file_bgl = ts_file_bgl.drop(parts_bgl.columns, axis=1)
    ts_file_gwl = ts_file_gwl.drop(parts_gwl.columns, axis=1)
    ts_file_gws = ts_file_gws.drop(parts_gws.columns, axis=1)

    print(stitch2_id)
    ts_file_bgl[f'{stitch2_id}_1'] = parts_bgl.bfill(axis=1).iloc[:, 0]
    ts_file_gwl[f'{stitch2_id}_1'] = parts_gwl.bfill(axis=1).iloc[:, 0]
    ts_file_gws[f'{stitch2_id}_1'] = parts_gws.bfill(axis=1).iloc[:, 0]

# stitching 2 last parts
stitch2_ids = ['KT053', 'RJ098', 'JA004', 'RJ073', 'SY014', 'SY104']
for stitch2_id in stitch2_ids:
    parts_bgl = ts_file_bgl.filter(like=stitch2_id, axis=1).iloc[:,1:]
    parts_gwl = ts_file_gwl.filter(like=stitch2_id, axis=1).iloc[:,1:]
    parts_gws = ts_file_gws.filter(like=stitch2_id, axis=1).iloc[:,1:]
    
    ts_file_bgl = ts_file_bgl.drop(parts_bgl.columns, axis=1)
    ts_file_gwl = ts_file_gwl.drop(parts_gwl.columns, axis=1)
    ts_file_gws = ts_file_gws.drop(parts_gws.columns, axis=1)

    print(stitch2_id)
    ts_file_bgl[f'{stitch2_id}_1'] = parts_bgl.bfill(axis=1).iloc[:, 0]
    ts_file_gwl[f'{stitch2_id}_1'] = parts_gwl.bfill(axis=1).iloc[:, 0]
    ts_file_gws[f'{stitch2_id}_1'] = parts_gws.bfill(axis=1).iloc[:, 0]


# splitting
split_ids = ['RJ031_0', 'DH001', 'SY078', 'SY027', 'RJ142', 'CM036_1', 'SY046_0', 'MY023_1', 
             'SY010', 'MY016', 'DI046_2', 'PA028', 'RJ102', 'RJ006', 'CM062', 'DH079', 'KT007_1',
             'DH090_2', 'KT034_1', 'PA018','TA033_0', 'FA007', 'DI008', 'BA021', 'DI033',
             'KT022', 'FA013', 'RJ077', 'DI042','DH069', 'JE030', 'RJ009', 'RJ033', 'MY087', 
             'RJ044', 'FA008', 'BA001', 'KT001', 'CT005', 'RJ103', 'JE017', 'RJ069', 'RJ068', 
             'PA011_0', 'JE015', 'CM057', 'RJ081', 'SY008', 'TA029', 'BA008_1', 'BA013_2', 
             'KH002_1', 'CT003_0', 'RJ088', 'DH076', 'MY051', 'BO035', 'FA004', 'BO027_1',
             'DI030', 'KT020', 'JE046', 'KH004', 'RA-43', 'DI076', 'MY018', 'KT029', 'DH073',
             'MY079', 'BO026', 'CT011', 'SY011', 'KH009', 'PA012', 'KT041', 'BO020',
             'MY073', 'CM016_1', 'BO006', 'PA031', 'MY053_1', 'JE039', 'SY055_1', 'DH005',
             'KHCP03', 'NA023_0', 'RJ029', 'BA005_2', 'MY074','NA015_1', 'CT004_1', 'RJ076_1']

split_dates = ['2010-01-01', '2012-01-01', '2007-01-01', '1984-01-01', '1992-01-01',
               '2001-01-01', '1992-01-01', '1990-01-01', '1985-01-01', '1991-01-01', 
               '1989-01-01', '2007-01-01', '1993-01-01', '1993-01-01', '1988-01-01', 
               '1991-06-01', '1982-01-01', '1990-01-01', '1989-01-01', '1986-01-01', 
               '1993-06-01', '1986-01-01', '1991-01-01', '1998-01-01', '1986-01-01', 
               '1993-01-01', '1983-01-01', '1986-06-01', '1992-01-01', '1990-01-01', 
               '1993-06-01', '1992-06-01', '1992-06-01', '1991-01-01', '1983-06-01', 
               '1986-06-01', '1985-01-01', '1987-01-01', '1986-06-01', '1992-01-01',
               '1991-01-01', '1982-06-01', '1992-06-01', '1992-01-01', '1984-01-01',
               '1990-06-01', '1992-01-01', '1986-06-01', '1984-06-01', '1992-01-01', 
               '1992-01-01', '1990-01-01', '1985-06-01', '1992-01-01', '1990-06-01', 
               '1987-06-01', '1987-01-01', '1986-01-01', '1989-01-01', '1993-01-01', 
               '1987-01-01', '2002-01-01', '1987-06-01', '1993-01-01', '1992-06-01',
               '1992-01-01', '1991-06-01', '1985-06-01', '1992-01-01', '1991-01-01',
               '1989-06-01', '1984-06-01', '1985-01-01', '1990-01-01', '1992-01-01',
               '1988-01-01', '1979-01-01', '1975-01-01', '2012-01-01', '1987-01-01',
               '1985-01-01', '1991-01-01', '2000-01-01', '1991-01-01', '1991-01-01',
               '2003-01-01', '1988-01-01', '2005-01-01', '2005-01-01', '1991-01-01', 
               '1988-01-01', '1991-01-01']

for i, split_id in enumerate(split_ids):
    entry_bgl = ts_file_bgl[split_id]
    entry_gwl = ts_file_gwl[split_id]
    entry_gws = ts_file_gws[split_id]
    ts_file_bgl = ts_file_bgl.drop(split_id, axis=1)
    ts_file_gwl = ts_file_gwl.drop(split_id, axis=1)
    ts_file_gws = ts_file_gws.drop(split_id, axis=1)
    
    mask = entry_bgl.index < split_dates[i]

    if split_id in ['MY073', 'CM016_1']:
        mask = entry_bgl.index > split_dates[i]

    if split_id in ['SY027', 'RJ142', 'SY046_0', 'MY023_1', 'SY010', 'MY016', 
                    'DI046_2', 'PA028', 'RJ102', 'RJ006', 'CM062', 'DH079', 'KT007_1', 
                    'DH090_2', 'KT034_1', 'PA018', 'TA033_0', 'FA007', 'DI008', 'BA021', 'DI033',
                    'KT022', 'FA013', 'RJ077', 'DI042', 'DH069', 'JE030', 'RJ009', 'RJ033', 'MY087',
                    'RJ044', 'FA008', 'BA001', 'KT001', 'CT005', 'RJ103', 'JE017', 'RJ069',
                    'RJ068', 'PA011_0', 'JE015', 'CM057', 'RJ081', 'SY008', 'TA029', 'BA008_1',
                    'BA013_2', 'KH002_1', 'CT003_0', 'RJ088', 'DH076', 'MY051', 'BO035', 'FA004',
                    'BO027_1', 'DI030', 'KT020', 'KH004', 'RA-43',
                    'DI076', 'MY018', 'KT029', 'DH073', 'MY079', 'BO026', 'CT011', 'SY011',
                    'KH009', 'PA012', 'KT041', 'BO020', 'BO006', 'PA031', 'MY053_1', 'JE039',
                    'SY055_1', 'DH005', 'RJ029', 'BA005_2', 'NA015_1', 'CT004', 'RJ076_1']:
        ts_file_bgl[f'{split_id}_0'] = entry_bgl.where(mask)
        ts_file_gwl[f'{split_id}_0'] = entry_gwl.where(mask)
        ts_file_gws[f'{split_id}_0'] = entry_gws.where(mask)
        ts_file_bgl[f'{split_id}_1'] = entry_bgl.where(~mask)
        ts_file_gwl[f'{split_id}_1'] = entry_gwl.where(~mask)
        ts_file_gws[f'{split_id}_1'] = entry_gws.where(~mask)
    else:
        ts_file_bgl[f'{split_id}'] = entry_bgl.where(mask)
        ts_file_gwl[f'{split_id}'] = entry_gwl.where(mask)
        ts_file_gws[f'{split_id}'] = entry_gws.where(mask)

# removing
del_ids = ['DH124S', 'KT031_1']
for del_id in del_ids:
    ts_file_bgl = ts_file_bgl.drop(del_id, axis=1)
    ts_file_gwl = ts_file_gwl.drop(del_id, axis=1)
    ts_file_gws = ts_file_gws.drop(del_id, axis=1)

/tmp/ipykernel_49139/3232304779.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  ts_file_bgl[stitch_id] = parts_bgl.bfill(axis=1).iloc[:, 0]
/tmp/ipykernel_49139/3232304779.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  ts_file_gwl[stitch_id] = parts_gwl.bfill(axis=1).iloc[:, 0]
/tmp/ipykernel_49139/3232304779.py:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=

PA020
DI051
SY055
SY048
CT022
NA006
RJ076
KT053


/tmp/ipykernel_49139/3232304779.py:44: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  ts_file_bgl[f'{stitch2_id}_1'] = parts_bgl.bfill(axis=1).iloc[:, 0]
/tmp/ipykernel_49139/3232304779.py:45: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  ts_file_gwl[f'{stitch2_id}_1'] = parts_gwl.bfill(axis=1).iloc[:, 0]
/tmp/ipykernel_49139/3232304779.py:46: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using

RJ098
JA004
RJ073
SY014
SY104


/tmp/ipykernel_49139/3232304779.py:122: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  ts_file_bgl[f'{split_id}_0'] = entry_bgl.where(mask)
/tmp/ipykernel_49139/3232304779.py:123: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  ts_file_gwl[f'{split_id}_0'] = entry_gwl.where(mask)
/tmp/ipykernel_49139/3232304779.py:124: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.

#### 4. Save preprocessed time series

In [9]:
ts_file_gws.to_csv(f'{root_path}/target/filtered_gws_ts_data_1961_2019.csv')
ts_file_gwl.to_csv(f'{root_path}/target/filtered_gwl_ts_data_1961_2019.csv')
ts_file_bgl.to_csv(f'{root_path}/target/filtered_bgl_ts_data_1961_2019.csv')